---
# ❗❗ 참고 사항
본인 PC에서는 돌아가지 않아서 지인의 PC에서 아래 이미지 증강 코드를 부탁해 돌렸습니다.

```python
import os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

ORIG_ROOT = r"D:\Downloads\resized_images"
AUG_ROOT  = r"D:\Downloads\augmented_resized_images"
n_aug_per_image = 2  # 원본 1장 당 증강본을 몇 개 생성할지

# ─── 2. ImageDataGenerator 설정 ────────────────────────────────────────────
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    brightness_range=[0.3, 1.0],
    zoom_range=0.3,           
    fill_mode="nearest"
)

os.makedirs(AUG_ROOT, exist_ok=True)

all_filenames = [
    fn for fn in os.listdir(ORIG_ROOT)
    if fn.lower().endswith((".jpg", ".jpeg", ".png"))
]

for fname in all_filenames:
    orig_path = os.path.join(ORIG_ROOT, fname)
    
    img = load_img(orig_path, target_size=(224, 224))
    x = img_to_array(img)  # (224, 224, 3), float32
    x = x.reshape((1,) + x.shape)  # (1, 224, 224, 3)

    # n_aug_per_image만큼 증강본 생성
    aug_count = 0
    for batch in datagen.flow(
        x,
        batch_size=1,
        save_to_dir=AUG_ROOT,
        save_prefix=f"aug_{os.path.splitext(fname)[0]}",
        save_format="jpg"
    ):
        aug_count += 1
        if aug_count >= n_aug_per_image:
            break

    # 원본+증강
    shutil.copy(orig_path, os.path.join(AUG_ROOT, fname))

print("증강 완료")
```
---

In [1]:
import pandas as pd
import os

In [ ]:
df=pd.read_csv("../../data/processed/02_processed_data/ALL_labels.csv")
AUG_DIR="../../data/processed/03_processed_data/resized_images_final"

In [ ]:
new_rows = []
for idx, row in df.iterrows():
    original_name=row["filename"]  # "F0001_IND_D_18_-45_01.JPG"
    base, _=os.path.splitext(original_name)  # "F0001_IND_D_18_-45_01"
    # 같은 폴더 내에서 aug_F0001_IND_D_18_-45_01_*.jpg 검색
    prefix=f"aug_{base}"
    for aug_fname in os.listdir(AUG_DIR):
        if aug_fname.startswith(prefix) and aug_fname.lower().endswith(".jpg"):
            # 증강본이 맞으면 원본 라벨과 동일한 딕셔너리로 새로운 행 생성
            new_row=row.copy()
            new_row["filename"]=aug_fname
            new_rows.append(new_row)

if new_rows:
    df_aug=pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
else:
    df_aug=df.copy()

df_aug.to_csv("../../data/processed/03_processed_data/ALL_labels_with_augmented.csv", index=False)
print(f"CSV 확장 완료:\n원본 {len(df)} -> 증강 후 {len(df_aug)} 행")

CSV 확장 완료:
원본 44151 -> 증강 후 132472 행
